In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
tokenizer = AutoTokenizer.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')
model = AutoModelForSequenceClassification.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')

In [2]:
import torch
import numpy as np

In [8]:
new_doc = """
//
//  GithubWebPresenter.swift
//  boostCamp
//
//  Created by godemodegame on 12.06.2021.
//

import Foundation

protocol GithubWebPresenterProtocol: AnyObject {
    func requestForCallbackURL(request: URLRequest)
    func viewLoaded()
}

final class GithubWebPresenter: GithubWebPresenterProtocol {

    // MARK: Vars

    weak var view: GithubWebViewProtocol?
    let githubService: GithubServiceProtocol

    var completion: (() -> Void)?

    // MARK: Lifecycle

    init(
        view: GithubWebViewProtocol,
        githubService: GithubServiceProtocol
    ) {
        self.view = view
        self.githubService = githubService
    }

    // MARK: - GithubWebPresenterProtocol

    func viewLoaded() {
        guard let request = githubService.makeUrl() else {
            fatalError("no url in github view")
        }
        view?.loadWebView(with: request)
    }

    // MARK: - тут начинается дичь, которую надо вынести в сервис, ибо ну не дела

    func requestForCallbackURL(request: URLRequest) {
        // Get the authorization code string after the '?code=' and before '&state='
        let requestURLString = (request.url?.absoluteString)! as String
        print(requestURLString)
        if requestURLString.hasPrefix(GithubConstants.REDIRECT_URI) {
            if requestURLString.contains("code=") {
                if let range = requestURLString.range(of: "=") {
                    let githubCode = requestURLString[range.upperBound...]
                    if let range = githubCode.range(of: "&state=") {
                        let githubCodeFinal = githubCode[..<range.lowerBound]
                        githubRequestForAccessToken(authCode: String(githubCodeFinal))
                        // Close GitHub Auth ViewController after getting Authorization Code
                        view?.close()
                    }
                }
            }
        }
    }

    func githubRequestForAccessToken(authCode: String) {
        let grantType = "authorization_code"
        // Set the POST parameters.
        let postParams = "grant_type=" + grantType + "&code=" + authCode + "&client_id=" + GithubConstants.CLIENT_ID + "&client_secret=" + GithubConstants.CLIENT_SECRET
        let postData = postParams.data(using: String.Encoding.utf8)
        let request = NSMutableURLRequest(url: URL(string: GithubConstants.TOKENURL)!)
        request.httpMethod = "POST"
        request.httpBody = postData
        request.addValue("application/json", forHTTPHeaderField: "Accept")
        let session = URLSession(configuration: URLSessionConfiguration.default)
        session.dataTask(with: request as URLRequest) { [weak self] (data, response, _) -> Void in
            let statusCode = (response as! HTTPURLResponse).statusCode
            if statusCode == 200 {
                let results = try! JSONSerialization.jsonObject(with: data!, options: .allowFragments) as? [AnyHashable: Any]
                let accessToken = results?["access_token"] as! String
                // Get user's id, display name, email, profile pic url
                self?.fetchGitHubUserProfile(accessToken: accessToken)
            }
        }.resume()
    }

    func fetchGitHubUserProfile(accessToken: String) {
        let tokenURLFull = "https://api.github.com/user"
        let verify: NSURL = NSURL(string: tokenURLFull)!
        let request: NSMutableURLRequest = NSMutableURLRequest(url: verify as URL)
        request.addValue("Bearer " + accessToken, forHTTPHeaderField: "Authorization")
        URLSession.shared.dataTask(with: request as URLRequest) { [weak self] data, _, error in
            if error == nil {
                let result = try! JSONSerialization.jsonObject(with: data!, options: .allowFragments) as? [AnyHashable: Any]
                if let urlString = result?["url"] as? String{
                    self?.fetchUser(from: urlString)
                }
                DispatchQueue.main.async {
                    self?.completion?()
                }
            }
        }.resume()
    }

    func fetchUser(from link: String) {
        // "https://github.com/godemodegame" "https://api.github.com/users/godemodegame"
        let potentialUsers = link.split(separator: "/").filter {
            $0 != "https:" && $0 !=  "api.github.com" && $0 != "github.com" && $0 != "users"
        }
        print(potentialUsers)
    }
}
"""

In [9]:
inputs = tokenizer(new_doc, return_tensors="pt", truncation=True, padding='max_length')
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [10]:
loss

tensor(0.0024, grad_fn=<NllLossBackward>)

In [11]:
logits

tensor([[-3.1823,  2.8317]], grad_fn=<AddmmBackward>)